In [152]:
import socket
from emoji import demojize
import logging
from datetime import datetime
import re, string
import pandas as pd, numpy as np
import datetime
from joblib import dump, load
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from IPython.display import display

In [153]:
server = 'irc.chat.twitch.tv'
port = 6667
nickname = 'vcManifest'
token = 'oauth:ys4f5zms68rnypcsp3wfqibu3tn6z2'
channel = '#'+input()

 cohhcarnage


In [154]:
sock = socket.socket()
sock.connect((server, port))
sock.send(f"PASS {token}\n".encode('utf-8'))
sock.send(f"NICK {nickname}\n".encode('utf-8'))
sock.send(f"JOIN {channel}\n".encode('utf-8'))

18

In [155]:
handlers = [logging.FileHandler('chat.log', encoding='utf-8')]
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s — %(message)s', datefmt='%Y-%m-%d, %H:%M:%S', handlers=handlers)

In [156]:
m_toxic,r_toxic = load('m_toxic.joblib')
m_severe,r_severe = load('m_severe.joblib')
m_obscene,r_obscene = load('m_obscene.joblib')
m_threat,r_threat = load('m_threat.joblib')
m_insult,r_insult = load('m_insult.joblib')
m_identity,r_identity = load('m_identity.joblib')

In [157]:
label_cols = ['datetime', 'channel', 'username', 'message', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'flag', 'reason']

In [158]:
def get_prediction(comment):
    #Vectorise
    res = vec.transform(comment)
    
    # Create Pred Array 
    pred = np.zeros((1, len(label_cols)-6))
    
    # Predict for each Model
    pred[:,0] = m_toxic.predict_proba(res.multiply(r_toxic))[:,1]
    pred[:,1] = m_severe.predict_proba(res.multiply(r_severe))[:,1]
    pred[:,2] = m_obscene.predict_proba(res.multiply(r_obscene))[:,1]
    pred[:,3] = m_threat.predict_proba(res.multiply(r_threat))[:,1]
    pred[:,4] = m_insult.predict_proba(res.multiply(r_insult))[:,1]
    pred[:,5] = m_identity.predict_proba(res.multiply(r_identity))[:,1]
    
    return pred

In [159]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()
vec = load('vectorizer_fit.joblib')

In [160]:
df = pd.DataFrame(columns = label_cols)

In [161]:
def flag(prediction):
    # Flag chat message with moderation.
    # Score weightings can be changed for harsher or softer moderation.
    toxicity_weight = 0.95
    threat_weight = 0.8
    insult_weight = 0.95
    identity_hate_weight = 0.75
    
    trigger = 'n/a'
    # 0 = No moderation
    # 1 = Timeout 
    # 2 = Banned
    flag_status = 'none'
    
    # If identity_hate score is greater than identity_hate then flag for moderation.
    if prediction[0][5] > identity_hate_weight:
        trigger = 'identity_hate'
        flag_status = 'banned'
    # If threat score is greater than threat_weight then flag for moderation.
    elif (prediction[0][3] > threat_weight):
        trigger = 'threat'
        flag_status = 'banned'
    # If toxicity and severe toxicty scores are greater than toxicity_weight, then flag message for moderation.
    elif (prediction[0][0] > toxicity_weight) and (prediction[0][1]> toxicity_weight):
        trigger = 'toxicity'
        flag_status = 'timedout'
    # If insult is greater than insult_weight then flag for moderation.
    elif prediction[0][4] > insult_weight:
        trigger = 'insult'
        flag_status = 'timedout'
        
    return flag_status, trigger

To see the messages in real-time open a new MSYS2 terminal, navigate to the log's location, and run **'tail -f chat.log'**.

In [162]:
while True:
    resp = sock.recv(2048).decode('utf-8')

    if resp.startswith('PING'):
        sock.send("PONG\n".encode('utf-8'))
    
    elif len(resp) > 0:
        try:
            current = datetime.datetime.now()
            date_time = current.strftime("%Y-%m-%d, %H:%M:%S")
            msg = demojize(resp)

            logging.info(msg)
            username, channel, message = re.search(':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', msg).groups()
            message = message.replace("\r","")
            res = get_prediction([message])
            res_flag = flag(res)
            data = {'datetime':date_time, 'channel':channel, 'username':username, 'message':message, 'toxic':res[0][0], 'severe_toxic':res[0][1], 'obscene':res[0][2], 'threat':res[0][3], 'insult':res[0][4], 'identity_hate':res[0][5], 'flag':res_flag[0],'reason':res_flag[1]}
            df.loc[len(df)] = data
            if df['flag'].loc[len(df)-1] != 'none':
                display(df.tail(1))
        except:
            pass

,datetime,channel,username,message,toxic,severe_toxic,obscene,threat,insult,identity_hate,flag,reason
754,"2021-09-23, 13:47:55",cohhcarnage,no1schmuck,Sleep ya bastard cohhOld,0.999974,0.091912,0.999673,0.003106,0.989928,0.015302,timedout,insult


,datetime,channel,username,message,toxic,severe_toxic,obscene,threat,insult,identity_hate,flag,reason
1738,"2021-09-23, 13:59:55",cohhcarnage,nuuriell,Loser.,0.998697,0.013367,0.02027,0.000662,0.988568,0.003027,timedout,insult


,datetime,channel,username,message,toxic,severe_toxic,obscene,threat,insult,identity_hate,flag,reason
2103,"2021-09-23, 14:04:01",cohhcarnage,maelintv,censored kill,0.952449,0.016273,0.039875,0.990389,0.023103,0.005178,banned,threat


,datetime,channel,username,message,toxic,severe_toxic,obscene,threat,insult,identity_hate,flag,reason
3164,"2021-09-23, 14:20:11",cohhcarnage,zkarlkani,Death to VIP cohhAim,0.920282,0.002082,0.00271,0.967998,0.005798,0.219293,banned,threat


,datetime,channel,username,message,toxic,severe_toxic,obscene,threat,insult,identity_hate,flag,reason
4732,"2021-09-23, 14:42:42",cohhcarnage,skinerch,they would kill you so...,0.972453,0.091481,0.0734,0.978118,0.028483,0.019369,banned,threat


...

,datetime,channel,username,message,toxic,severe_toxic,obscene,threat,insult,identity_hate,flag,reason
5594,"2021-09-23, 14:52:55",cohhcarnage,zeannishneer,"dirty, dirty cheater NOPERS",0.964705,0.003741,0.55164,0.0001,0.950892,0.048569,timedout,insult


,datetime,channel,username,message,toxic,severe_toxic,obscene,threat,insult,identity_hate,flag,reason
5652,"2021-09-23, 14:53:37",cohhcarnage,vvardy,Goddamn,0.999891,0.00033,0.998477,0.000339,0.968558,0.000226,timedout,insult


,datetime,channel,username,message,toxic,severe_toxic,obscene,threat,insult,identity_hate,flag,reason
6057,"2021-09-23, 14:58:51",cohhcarnage,artboyo,these cops suck,0.999972,0.060472,0.999156,0.000129,0.970895,0.001324,timedout,insult


,datetime,channel,username,message,toxic,severe_toxic,obscene,threat,insult,identity_hate,flag,reason
8539,"2021-09-23, 18:47:55",cohhcarnage,b0kkr,kick his ass!,0.999997,0.006051,0.998851,0.099263,0.998946,0.066726,timedout,insult


KeyboardInterrupt: 